In [1]:
import pandas as pd

In [3]:
from pathlib import Path

import sequence_sensei
import numpy as np

In [4]:
import axelrod as axl

In [5]:
axl.__version__

'4.2.0'

**Outputs**

In [6]:
dfs = []
for file in Path("raw_data/").glob("*.csv"):
    dfs.append(pd.read_csv(file, index_col=0,))

In [7]:
df = pd.concat(dfs).reset_index(drop=True)

In [8]:
df = df.drop(columns=['index'])

In [9]:
df = df[~(df['opponent'].str.contains('Majority'))]

**Inputs**

In [11]:
import axelrod as axl

In [12]:
strategies = {strategy().name: strategy() for strategy in axl.all_strategies}

In [13]:
opponents_moves = []
for opponent in df['opponent']:
    
    opponent = strategies[opponent.split(':')[0]]
    
    sequence = sequence_sensei.get_sequence_str(df.iloc[0].values[-205:])
    cycler = axl.Cycler(sequence)

    match = axl.Match([opponent, cycler], turns=205)
    _ = match.play()
    
    moves_opp, moves_cycler = zip(*match.result)
    
    assert sequence_sensei.get_sequence_str(moves_cycler) == sequence
    
    opponents_moves.append((opponent,) + moves_opp)

In [15]:
len(opponents_moves)

3688

**Transform Data for RNN**

In [17]:
sequences = [[moves[0]] + [move.value for move in moves[1:]] for moves in opponents_moves]

In [18]:
data = pd.DataFrame(sequences)

In [19]:
data.columns = df.columns

In [20]:
data.to_csv('sequences.csv'), df.to_csv('targets.csv')

(None, None)

**Transform the data**

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
inputs = pd.read_csv('sequences.csv', index_col=0)
outputs = pd.read_csv('targets.csv', index_col=0)

In [4]:
input_train, input_test, output_train,  output_test = train_test_split(inputs, outputs,
                                                                       test_size=0.33,
                                                                       random_state=0)

In [5]:
def get_batches(X, Y):
    inputs, outputs = [], []
    for max_length in range(1, 203):
        n_sequences = len(X)
        for n in range(n_sequences):
            x = X.iloc[n].values[:max_length]

            y = Y.iloc[n].values[1 :max_length + 1] 

            inputs.append(x), outputs.append(y)
    return inputs, outputs

In [6]:
label = 'train'

In [7]:
len(input_test), len(output_test)

(1218, 1218)

In [8]:
inputs, outputs = get_batches(input_train.drop(columns=['opponent']),
                              output_train.drop(columns=['opponent']))

In [ ]:
inpts = pd.DataFrame(inputs)

In [10]:
inpts.to_csv('inputs_%s.csv' % label)

In [9]:
outpts = pd.DataFrame(outputs)

In [10]:
outpts.to_csv('outputs_%s.csv' % label)